# Aplicación de resumenes con LangChain y OpenAI

## Preparando los Datos

In [4]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv, find_dotenv

In [6]:
load_dotenv(find_dotenv(), override=True)

True

# Resumen Básico

In [ ]:
texto = """
La inteligencia artificial (IA), en el contexto de las 
ciencias de la computación, es una disciplina y un conjunto 
de capacidades cognoscitivas e intelectuales expresadas por 
sistemas informáticos o combinaciones de algoritmos cuyo propósito 
es la creación de máquinas que imiten la inteligencia humana para 
realizar tareas, y que pueden mejorar conforme recopilan información
. A diferencia de la inteligencia sintética, la inteligencia 
artificial no tiene como finalidad reemplazar a los humanos, 
sino mejorar significativamente las capacidades y contribuciones 
de estos. Se hizo presente poco después de la Segunda Guerra 
Mundial con el desarrollo de la «prueba de Turing», mientras 
que la locución fue acuñada en 1956 por el informático John 
McCarthy en la Conferencia de Dartmouth.

En la actualidad, la inteligencia artificial abarca una 
gran variedad de subcampos. Éstos van desde áreas de propósito 
general, aprendizaje y percepción, a otras más específicas como 
el reconocimiento de voz, el juego de ajedrez, la demostración 
de teoremas matemáticos, la escritura de poesía y el diagnóstico 
de enfermedades. La inteligencia artificial sintetiza y automatiza 
tareas que en principio son intelectuales y, por lo tanto, es 
potencialmente relevante para cualquier ámbito de diversas 
actividades intelectuales humanas. En este sentido, es un campo 
genuinamente universal.3

La arquitectura de las inteligencias artificiales y los procesos 
por los cuales aprenden, se mejoran y se implementan en algún área 
de interés varían según el enfoque de utilidad que se les quiera 
dar, pero de manera general, estos van desde la ejecución de 
sencillos algoritmos hasta la interconexión de complejas redes 
neuronales artificiales que intentan replicar los circuitos 
neuronales del cerebro humano y que aprenden mediante diferentes 
modelos de aprendizaje tales como el aprendizaje automático, el 
aprendizaje por refuerzo, el aprendizaje profundo o el aprendizaje 
supervisado.

Por otro lado, el desarrollo y aplicación de la inteligencia 
artificial en muchos aspectos de la vida cotidiana también ha 
propiciado la creación de nuevos campos de estudio como la robótica 
y la ética de las máquinas que abordan aspectos relacionados con 
la ética en la inteligencia artificial y que se encargan de 
analizar cómo los avances en este tipo de tecnologías impactarían
en diversos ámbitos de la vida, así como el manejo responsable y 
ético que se les debería dar a los mismos, además de establecer 
cuál debería ser la manera correcta de proceder de las máquinas y 
las reglas que deberían cumplir.

En cuanto a su clasificación, tradicionalmente se divide a la 
inteligencia artificial en inteligencia artificial débil, la cual
es la única que existe en la actualidad y que se ocupa de realizar 
tareas específicas, e inteligencia artificial fuerte, que sería 
una IA que excediese las capacidades humanas. Algunos expertos 
creen que si alguna vez se alcanza este nivel, se podría dar 
lugar a la aparición de una singularidad tecnológica, es decir, 
una entidad tecnológica superior que se mejoraría a sí misma 
constantemente, volviéndose incontrolable para los humanos, 
dando pie a teorías como el basilisco de Roko.

Algunas de las inteligencias artificiales más conocidas y 
utilizadas en la actualidad alrededor del mundo incluyen 
inteligencia artificial en el campo de la salud, asistentes 
virtuales como Alexa, el asistente de Google o Siri, traductores 
automáticos como el traductor de Google y DeepL, sistemas de 
recomendación como el de la plataforma digital de YouTube, motores 
de ajedrez y otros juegos como Stockfish y AlphaZero, chatbots 
como ChatGPT, creadores de arte de inteligencia artificial como 
Midjourney, Dall-e y Stable Diffusion, e incluso la conducción de 
vehículos autónomos como Tesla Autopilot.
"""

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
mensajes = [
    SystemMessage(content='Usted es un "copywriter" experto con gran experiencia en resumir documentos'),
    HumanMessage(content=f'haz un resumen breve y conciso del siguiente texto:\n Texto: {texto}')
]

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [ ]:
llm.get_num_tokens(texto)

In [ ]:
resumen = llm(mensajes)

In [ ]:
print(resumen)

In [ ]:
print(resumen.content)

## Usando Plantillas de Promts (Prompt Templates)

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
template = '''
Escriba un resumen conciso y breve del siguiente texto:
TEXTO: {texto}
Traduzca el resumen al {idioma}.
'''

In [ ]:
prompt = PromptTemplate(
    input_variables=['texto', 'idioma'],
    template=template
)

In [ ]:
llm.get_num_tokens(prompt.format(texto=texto, idioma='Inglés'))

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
resumen = chain.run({'texto': texto, 'idioma':'Español'})

In [ ]:
print(resumen)

## Resumenes de documentos usando SuffDocumentChain

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [ ]:
with open('minecraft.txt', encoding='utf-8') as f:
    texto = f.read()

print(texto)

In [ ]:
llm.get_num_tokens(texto)

In [ ]:
docs = [Document(page_content=texto)]
llm = ChatOpenAI(temperature=0, model_name='gpt-4')

In [ ]:
template = '''
Escriba un resumen conciso y breve del siguiente texto:
TEXTO: {text}.
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
resumen = chain.run(docs)

In [ ]:
print(resumen)

## Resume Documentos largos usando map_reduce

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
with open('ia.txt', encoding='utf-8') as f:
    texto = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [ ]:
llm.get_num_tokens(texto)

In [ ]:
len(texto)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=12000, chunk_overlap=100)
chunks = text_splitter.create_documents([texto])

In [ ]:
len(chunks)

In [ ]:
print(chunks[4].page_content)

In [ ]:
for i in range (0, 5):
    print(llm.get_num_tokens(chunks[i].page_content))

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
resumen = chain.run(chunks)

In [ ]:
print(resumen)

##  map_reduce con Prompts Personalizados

In [ ]:
map_prompt = '''
Escriba un resumen conciso y breve del siguiente texto:
Que el resumen esté en español.
TEXTO: {text}
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [ ]:
prompt_combinado = '''
Escriba un resumen conciso del siguiente texto que abarque los puntos claves.
Añade un título al resumen.
Comience su resumen con un PÁRRAFO DE INTRODUCCIÓN que ofrezca una visión general del tema SEGUIDO
por los PUNTOS tipo "BULLET Point" si es posible Y termine el resumen con una CONCLUSIÓN.
Texto: {text}
'''
prompt_template_combinado = PromptTemplate(template=prompt_combinado, input_variables=['text'])

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4')

In [ ]:
cadena_resumen = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=prompt_template_combinado,
    verbose=False
)
resumen = cadena_resumen.run(chunks)

In [ ]:
print(resumen)

### Resumenes usando refine Chain 

In [ ]:
pip install unstructured -q

In [ ]:
pip install pdf2image

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
loader = UnstructuredPDFLoader('ia.pdf')
data = loader.load()

In [6]:
print(data[0].page_content)

Inteligencia artificial

La inteligencia artificial (IA), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilan información.12 A diferencia de la inteligencia sintética, la inteligencia artificial no tiene como finalidad reemplazar a los humanos, sino mejorar significativamente las capacidades y contribuciones de estos. Se hizo presente poco después de la Segunda Guerra Mundial con el desarrollo de la «prueba de Turing», mientras que la locución fue acuñada en 1956 por el informático John McCarthy en la Conferencia de Dartmouth.

En la actualidad, la inteligencia artificial abarca una gran variedad de subcampos. Éstos van desde áreas de propósito general, aprendizaje y percepción, a otras más específicas como el 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=12000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [8]:
len(chunks)

5

In [9]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [10]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=False
)
resumen = chain.run(chunks)

In [11]:
print(resumen)

Artificial intelligence (AI) is a discipline that aims to create machines capable of imitating human intelligence and improving their performance through learning and gathering information. It encompasses various subfields and applications, such as speech recognition, chess playing, disease diagnosis, and virtual assistants. The development of AI has led to the emergence of new fields of study, including roboethics and machine ethics, which analyze the ethical implications and responsible use of AI technologies. AI can be classified into weak AI, which focuses on specific tasks, and strong AI, which exceeds human capabilities. The history of AI dates back to ancient Greek philosophers and has seen significant advancements since the 1950s. The field has experienced notable milestones, such as the development of the first programming language for problem-solving, the introduction of the term "artificial intelligence," and the creation of systems like the LogicTheorist and General Problem

## Refine con prompt personalizados

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [33]:
loader = UnstructuredPDFLoader('FIFA 23 Modo Carrera.pdf')
data = loader.load()

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [35]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4')

In [36]:
prompt_template = """Escribe un resumen conciso de lo siguiente extrayendo la información clave:
Text: {text}
Resumen:"""
prompt_inicial = PromptTemplate(template=prompt_template, input_variables=['text'])

In [37]:
refine_template = '''
    Tu trabajo consiste en elaborar un resumen final.
    He proporcionado un resumen existente hasta cierto punto: {existing_answer}.
    Por favor, perfeccione el resumen existente con algo más de contexto a continuación.
    ------------
    {text}
    ------------
    Comience el resumen final con una "Introducción" que ofrezca una visión general del tema seguido
    por los puntos más importantes ("Bullet Ponts"). Termina el resumen con una conslusión.
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text'])

In [38]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=prompt_inicial,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
resumen = chain.run(chunks)

In [39]:
print(resumen)

Introducción:
El episodio sigue la trayectoria del equipo de Colo-Colo en la serie FIFA 23, donde se esfuerzan por mejorar su posición en la tabla. Después de una victoria contra la Universidad de Chile y una derrota contra Audax, el equipo se enfrenta a Cobresal y la Unión. El jugador analiza la formación del equipo, realiza cambios estratégicos y se incorporan nuevas fuerzas básicas. El episodio concluye con la anticipación de los próximos partidos y la esperanza de una mejor actuación.

Puntos más importantes:
- Colo-Colo se encuentra en la cuarta posición después de su victoria contra la Universidad de Chile.
- El equipo se enfrenta a Cobresal en el próximo partido, que también tiene siete puntos.
- El jugador decide simular el partido contra Cobresal, pero Colo-Colo sufre una derrota.
- Se realiza el sorteo de los grupos de la Copa Libertadores, y Colo-Colo se encuentra en el mismo grupo que en la realidad.
- Se incorporan nuevas fuerzas básicas al equipo, incluyendo a Sergio Orel

## Resume usando Agentes de LangChains

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [20]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4')
wikipedia = WikipediaAPIWrapper()

In [23]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Útil para cuando necesitas obtener información de wikipedia sobre un solo tema"
    )
]

In [24]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [26]:
respuesta = agent_executor.run('¿Podría resumir brevemente la copa del mundo qatar 2022, quién fue el campeón y por cuánto ganaron?')



> Entering new  chain...
No tengo la información actualizada sobre la Copa del Mundo Qatar 2022. Necesito buscar en Wikipedia para obtener la información.
Action: Wikipedia
Action Input: Copa del Mundo de la FIFA Qatar 2022
Observation: Page: 2022 FIFA World Cup squads
Summary: The 2022 FIFA World Cup was an international football tournament held in Qatar from 20 November to 18 December 2022. The 32 national teams involved in the tournament were required to register a squad of up to 26 players, including three goalkeepers. Only players in these squads were eligible to take part in the tournament.A provisional release list of between 35 and 55 players per national team was submitted to FIFA by 21 October 2022, one month prior to the opening match of the tournament. The release lists were not made public by FIFA. From the preliminary squad, the final list of at most 26 and at least 23 players per national team were submitted to FIFA by 14 November, 19:00 AST (UTC+3), six days prior to 

Thought:No pude encontrar la información sobre el marcador final del partido. Intentaré buscar de nuevo.
Action: Wikipedia
Action Input: Final Copa del Mundo 2022 marcador
Observation: Page: Ecuador national football team
Summary: The Ecuador national football team (Spanish: Selección de fútbol de Ecuador) represents Ecuador in men's international football and is controlled by the Ecuadorian Football Federation (FEF). They joined FIFA in 1926 and CONMEBOL a year later.
Discarding an invitation to participate in the inaugural FIFA World Cup held in Uruguay, Ecuador did not make their tournament debut until 2002. After finishing above Brazil and Uruguay in the standings, the qualifying campaign marked the emergence of several players, such as Agustín Delgado, Álex Aguinaga, Iván Hurtado, Ulises de la Cruz and Iván Kaviedes, who would set the stage for Ecuador's achievements in the next decade. Having reached the Round of 16 in a memorable 2006 World Cup campaign, they were expected to de

In [27]:
print(respuesta)

La Copa del Mundo de la FIFA 2022 se llevó a cabo en Qatar del 20 de noviembre al 18 de diciembre de 2022. El campeón fue Argentina. No pude encontrar la información sobre el marcador final del partido.
